In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from graph_ex import SupplyChain
from dataclasses import dataclass
from numba import jit, njit


In [2]:
np.set_printoptions(suppress=True)

In [79]:
xls = pd.ExcelFile('data_ex.xlsx')
demand_dict = pd.read_excel(xls, sheet_name='demand', index_col=0, usecols="A:D", nrows=33).to_dict('index')  
prices_dict = pd.read_excel(xls, sheet_name = 'prices', index_col=0, usecols="A:D", nrows=33).to_dict('index') 
custs = pd.read_excel(xls, sheet_name='customers', index_col=0, usecols="A:C", nrows=33).to_dict('index')
farms_dict = pd.read_excel(xls, sheet_name='farms', index_col=0, usecols="A:F", nrows=31).to_dict('index')
products_dict = {'P1': 6, 'P2': 10, 'P3': 12}
transport_cost_per_egg = {'same_loc':0.10, 'different_loc':0.15}


def sum_to_num(sum_to, nums):
    ''' Generates random numbers that sum up to a given value
        Parameters:
        ----------
        sum_to: The values the numbers should sum up to.
        nums: The number of random numbers.
        Source: http://sunny.today/generate-random-integers-with-fixed-sum/
    '''
    return np.random.multinomial(sum_to, np.ones(nums)/nums, size=1)[0]



@dataclass
class SupplyChain():
    farms: dict
    customers: dict
    products: dict
    demand: dict
    prices: dict
    transport: dict
    graph: nx.DiGraph = nx.DiGraph()

    
    def edges_fprod_cprod(self) -> list:
        ''' Creates tuples of customers-products &
            farm-products for edges '''
        prod_edges = []
        for customer in self.customers.keys():
            for product in self.products.keys():
                for farm in self.farms.keys():
                    for f_prod in self.farms[farm]['Products']:
                        if product == f_prod:
                            prod_edges.append((farm + "_" + f_prod, customer + "_" + product))
        return prod_edges
    
    def set_demand_quantity(self): 
        ''' Adds node attribute -> demand and fills it with demand value from dict'''
        for customer in self.customers.keys():
            for product in self.products.keys():
                self.graph.nodes[customer + "_" + product]['demand'] = self.demand[customer][product]

    # def set_demand_quantity(self):
    #     ''' Set the demand quantity on the edge between cprod and customer node'''
    #     for customer in self.customers.keys():
    #         for product in self.products.keys():
    #             self.graph[customer+"_"+product][customer]['demand'] = self.demand[customer][product]

    # Buy prices on nodes
    def set_buying_prices(self):
        ''' Adds node attribute -> price and fills it with price values from dict'''
        for customer in self.customers.keys():
            for product in self.products.keys():
                self.graph.nodes[customer + "_" + product]['price'] = self.prices[customer][product]
    
    # Buy price on edge between cprod and customer
    # def set_buying_prices(self):
    #     ''' Set the buying price on the edge between cprod and customer node'''
    #     for customer in self.customers.keys():
    #         for product in self.products.keys():
    #             self.graph[customer + "_" + product][customer]['price'] = self.prices[customer][product]


    def set_supply_quantity(self):
        ''' Adds eggs_supply node attribute for farms and fills it with farm Qty values from dict'''
        for farm in self.farms.keys():
            self.graph.nodes[farm]['eggs_supply'] = self.farms[farm]['Qty']
    
    # def set_supply_price(self):
    #     ''' Adds cost_per_egg node attribute for farms and fills it with Cost values from dict '''
    #     for farm in self.farms.keys():
    #         self.graph.nodes[farm]['cost_per_egg'] = self.farms[farm]['Cost']

    def set_supply_cost(self):
        ''' Adds the cost_per_egg for edge between farm and fprods'''
        for farm in self.farms.keys():
            for fprod in self.graph.successors(farm):
                for cprod in self.graph.successors(fprod):
                    self.graph[fprod][cprod]['cost_per_egg'] = self.farms[farm]['Cost']        

    def set_fprod_locations(self):
        ''' Sets the location of the f_prod to the farm location '''
        for farm in self.farms.keys():
            for product in self.farms[farm]['Products']:
                self.graph.nodes[farm + "_" + product]['Location'] = self.farms[farm]['Location']

    def set_cprod_locations(self):
        for cust in self.customers.keys():
            for prod in self.products.keys():
                self.graph.nodes[cust + "_" + prod]['Location'] = self.customers[cust]['Location']

    def set_transport_costs(self):
        ''' Set the transport costs on the edges between fprod and cprod based on their locations'''
        # f_prods = [farm + "_" + product for farm in self.farms.keys() for product in self.farms[farm]['Products']]
        for farm in self.farms.keys():
            for fprod in self.graph.successors(farm):
                for cprod in self.graph.successors(fprod):
                    if self.graph.nodes[fprod]['Location'] == self.graph.nodes[cprod]['Location']:
                        self.graph[fprod][cprod]['transport_cost'] = self.transport['same_loc']
                    else:
                        self.graph[fprod][cprod]['transport_cost'] = self.transport['different_loc']
                        
    def set_eggs_packs(self):
        ''' Sets the eggs per pack on fprod and farm edge'''
        for farm in self.farms.keys():
            for product in self.farms[farm]['Products']:
                self.graph.nodes[farm + "_" + product]['eggs_per_pack'] = self.products[product]



    def __post_init__(self):

        # Fix the products list
        for farm in self.farms.keys():
            self.farms[farm]['Products'] = self.farms[farm]['Products'].split(', ')
        # Add farm nodes and connect to f_prods
        self.graph.add_edges_from([(farm, farm + "_" + product) for farm in self.farms.keys() 
                                                                for product in self.farms[farm]['Products']])      
        # Add customer nodes and connect to c_prods
        self.graph.add_edges_from([(cust + "_" + prod, cust) for cust in self.customers.keys() 
                                                            for prod in self.products.keys()])
        # connect f_prod & c_prod
        self.graph.add_edges_from(self.edges_fprod_cprod())

        self.set_demand_quantity()
        self.set_buying_prices()
        self.set_supply_quantity()
        self.set_supply_cost()
        self.set_fprod_locations()
        self.set_cprod_locations()
        self.set_transport_costs()
        self.set_eggs_packs()

    # Get demand values from nodes
    def get_demand_vec(self) -> np.ndarray:
        ''' Gets the demand node attributes from the graph as a dict 
            converts the dict's values to an array'''
        return np.fromiter(nx.get_node_attributes(self.graph,'demand').values(), dtype=np.int64)

    # # Get demand value from edges
    # def get_demand_vec(self) -> np.ndarray:
    #     ''' Returns an array with the demand values between cprod and customer edges'''
    #     return np.array([self.graph[cprod][customer]['demand'] 
    #     for customer in self.customers.keys() 
    #     for cprod in self.graph.predecessors(customer)]).astype(np.int64)
    

    def get_price_per_product(self) -> np.ndarray:
    #     return np.fromiter(nx.get_node_attributes(self.graph,'price').values(), dtype=np.float64)
        prices = []
        for farm in self.farms.keys():
            for fprod in self.graph.successors(farm):
                for cprod in self.graph.successors(fprod):
                    prices.append(self.graph.nodes[cprod]['price'])
        return np.array(prices).astype(np.float64)


    def get_eggs_supplied(self) -> np.ndarray:
        return np.fromiter(nx.get_node_attributes(self.graph,'eggs_supply').values(), dtype=np.int64)

    # def get_supply_costs(self) -> np.ndarray:
    #     ''' Gets the cost per egg supplied from farm to customer'''
    #     costs = []
    #     for farm in self.farms.keys():
    #         for prod in self.farms[farm]['Products']:
    #             fprod = farm + "_" + prod
    #             for _ in self.graph.successors(fprod):
    #                 costs.append(self.farms[farm]['Cost'])
    #     return np.array(costs).astype(np.float16)
    
    def get_supply_costs(self) -> np.ndarray:
        ''' Gets the cost per egg supplied from fprod to cprod'''
        supply_costs =[]
        for farm in self.farms.keys():
            for fprod in self.graph.successors(farm):
                for cprod in self.graph.successors(fprod):
                    supply_costs.append(self.graph[fprod][cprod]['cost_per_egg'])
        return np.array(supply_costs).astype(np.float64)

    # def get_transport_cost(self):
    #     ''' Gets the transport costs per egg supplied from farm to customer'''
    #     costs =[]
    #     for farm in self.farms.keys():
    #         for prod in self.farms[farm]['Products']:
    #             fprod = farm + "_" + prod
    #             for cust_prod in self.graph.successors(fprod):
    #                 if self.graph.nodes[cust_prod]['Location'] == self.graph.nodes['F1_P1']['Location']:
    #                     transport_cost = self.transport['same_loc']
    #                 else:
    #                     transport_cost = self.transport['different_loc']
    #                 costs.append(transport_cost)
    #     return np.array(costs).astype(np.float16)
    
    def get_transport_cost(self) -> np.ndarray:
        ''' Returns an array with the transport costs of the fprod to cprod'''
        transport_costs = []
        for farm in self.farms.keys():
            for fprod in self.graph.successors(farm):
                for cprod in self.graph.successors(fprod):
                    transport_costs.append(self.graph[fprod][cprod]['transport_cost'])
        return np.array(transport_costs).astype(np.float16)

    def get_optimising_tuples(self) -> list:
        ''' Return the tuples of fprod & cprod that need to be optimised'''
        return [(fprod, cprod) for farm in self.farms.keys() 
                for fprod in self.graph.successors(farm) 
                for cprod in self.graph.successors(fprod)]
    
    def get_indices_dict(self) -> dict:
        ''' Return a dict with the optimising tuples as keys and 
            their index position in the list of tuples as value '''
        return {item: idx for idx, item in enumerate(self.get_optimising_tuples())}

    def get_indices(self, fprod_cprod:str):
        ''' Takes an fprod or cprod value and returns all unique values (index) 
            from a dict of index values '''
        index_dict= self.get_indices_dict() 
        index_values = set()
        for i in range(len(index_dict)):
            for fprod, cprod in index_dict.keys():
                if fprod_cprod == fprod or fprod_cprod == cprod:
                    index_values.add(index_dict.get((fprod, cprod)))
        return list(index_values)
    
    def get_supply_check_variables(self) -> tuple:
        farm_wise_fprod_list = [[fprod for fprod in self.graph.successors(farm)] 
                                for farm in self.farms.keys()]
        packs = np.array([self.graph.nodes[fprod]['eggs_per_pack'] 
                            for farms in farm_wise_fprod_list for fprod in farms])
        lens = [len(farm) for farm in farm_wise_fprod_list]
        split_list = np.cumsum(lens)
        all_inds = np.vstack([self.get_indices(fprod) 
                    for farm in self.farms.keys() 
                    for fprod in self.graph.successors(farm)])    
        return all_inds, split_list, packs

    def get_demand_check_variables(self) -> tuple:
        ''' Return a list of index position where to split the list of all index values of cprods
        - lens: lengths of the list of index values for cprods
        - split_list: a cumulative sum of index values of the lengths
        - all_inds: all index values of cprods in a single array '''
        lens = np.array([len(self.get_indices(cprod)) 
                    for cust in self.customers.keys() 
                    for cprod in self.graph.predecessors(cust)]).astype(np.int64)
        split_list = np.cumsum(lens)

        all_inds = np.concatenate([self.get_indices(cprod) 
                    for customer in self.customers.keys() 
                    for cprod in self.graph.predecessors(customer)])
        return split_list, all_inds

    def split_eggs_supply_randomly(self) -> list:
        ''' Retuns a list of arrays with the total quantity supplied randomly distributed 
            among the total number of products '''
        dims = np.sum([len(self.farms[farm]['Products']) for farm in self.farms.keys()])
        z_vec = np.zeros(dims)
        split_indices = np.cumsum([len(self.farms[farm]['Products']) for farm in self.farms.keys()])
        split_vec = np.split(z_vec, split_indices)[:-1]  # Pop the last one -> its empty
        return [sum_to_num(supply, len(sv)) for sv, supply in zip(split_vec, self.get_eggs_supplied())]

    def pack_random_eggs(self) -> np.ndarray:
        ''' For every fprod it packs the available supply of eggs which have been
            split randomly across the fprods'''
        packs = []
        fprods = [self.farms[farm]['Products'] for farm in self.farms.keys()]
        for fprod, randegg in zip(fprods, self.split_eggs_supply_randomly()):
            for fp, re in zip(fprod, randegg):
                packs.append(np.floor(re/self.products[fp]))
        return np.array(packs).astype(np.int64)


In [80]:
sc = SupplyChain(farms=farms_dict, customers=custs, 
    products=products_dict, demand=demand_dict, 
    prices=prices_dict, transport=transport_cost_per_egg)

In [5]:
def sum_to_num(sum_to, nums):
    ''' Generates random numbers that sum up to a given value
        Parameters:
        ----------
        sum_to: The values the numbers should sum up to.
        nums: The number of random numbers.
        Source: http://sunny.today/generate-random-integers-with-fixed-sum/
    '''
    return np.random.multinomial(sum_to, np.ones(nums)/nums, size=1)[0]


In [6]:
def split_eggs_supply_randomly() -> list:
    ''' Retuns a list of arrays with the total quantity supplied distributed among the total number of products '''
    dims = np.sum([len(sc.farms[farm]['Products']) for farm in sc.farms.keys()])
    z_vec = np.zeros(dims)
    split_indices = np.cumsum([len(sc.farms[farm]['Products']) for farm in sc.farms.keys()])
    split_vec = np.split(z_vec, split_indices)[:-1]  # Pop the last one -> its empty
    return [sum_to_num(supply, len(sv)) for sv, supply in zip(split_vec, sc.get_eggs_supplied())]

In [7]:
def pack_random_eggs() -> list:
    packs = []
    fprods = [sc.farms[farm]['Products'] for farm in sc.farms.keys()]
    for fprod, randegg in zip(fprods, split_eggs_supply_randomly()):
        for fp, re in zip(fprod, randegg):
            packs.append(np.floor(re/sc.products[fp]))
    return np.array(packs).astype(np.int64)

In [8]:
all_tuples = [(fprod, cprod) for farm in sc.farms.keys() for fprod in sc.graph.successors(farm) for cprod in sc.graph.successors(fprod)]

In [9]:
eggs_supply = sc.get_eggs_supplied()

In [10]:
all_tuples_index_dict = {item: idx for idx, item in enumerate(all_tuples)}

vec = np.arange(585)

In [11]:
def get_index_from_dict(index_dict:dict, fprod_cprod:str):
    ''' Takes an fprod or cprod value and returns all unique values (index) 
        from a dict of index values '''
    index_values = set()
    for i in range(len(index_dict)):
        for fprod, cprod in index_dict.keys():
            if fprod_cprod == fprod or fprod_cprod == cprod:
                index_values.add(index_dict.get((fprod, cprod)))
    return list(index_values)

In [12]:
# def get_supply_check_variables(all_tuples_index_dict):
#     ''' Gets the indices & pack size of fprods in the list of tuples to be optimised
#         farm_wise_fprod_list: list of list of fprods by farms
#         packs: return the eggs_per_pack for the fprod in the farm_wise_fprod_list
#         indices: returns the index positions per farm in the farm_wise_fprod_list '''
#     farm_wise_fprod_list = [[fprod for fprod in sc.graph.successors(farm)] for farm in sc.farms.keys()]
#     packs = np.array([sc.graph.nodes[fprod]['eggs_per_pack'] for farms in farm_wise_fprod_list for fprod in farms])
#     indices = np.array([get_index_from_dict(all_tuples_index_dict, fprod) for farms in farm_wise_fprod_list for fprod in farms]).astype(np.int64)
#     return indices, packs
    

In [13]:
def get_supply_check_variables(all_tuples_index_dict) -> tuple:
    farm_wise_fprod_list = [[fprod for fprod in sc.graph.successors(farm)] for farm in sc.farms.keys()]
    packs = np.array([sc.graph.nodes[fprod]['eggs_per_pack'] for farms in farm_wise_fprod_list for fprod in farms])
    lens = [len(farm) for farm in farm_wise_fprod_list]
    split_list = np.cumsum(lens)
    all_inds = np.vstack([get_index_from_dict(all_tuples_index_dict, fprod) 
                for farm in sc.farms.keys() 
                for fprod in sc.graph.successors(farm)])    
    
    return all_inds, split_list, packs

In [14]:
sup_indices, sup_split_list, sup_packs = get_supply_check_variables(all_tuples_index_dict)

In [15]:
def get_demand_check_variable(all_tuples_index_dict):
    
    lens = np.array([len(get_index_from_dict(all_tuples_index_dict, cprod)) 
                for cust in sc.customers.keys() 
                for cprod in sc.graph.predecessors(cust)]).astype(np.int64)
    split_list = np.cumsum(lens)

    all_inds = np.concatenate([get_index_from_dict(all_tuples_index_dict, cprod) 
                for customer in sc.customers.keys() 
                for cprod in sc.graph.predecessors(customer)])
    
    
    return all_inds, split_list

In [16]:

demand = sc.get_demand_vec()

demand_indices, split_list = get_demand_check_variable(all_tuples_index_dict)


In [17]:
@njit
def feasible_vec(vec:np.ndarray)-> bool:
    global sup_indices, sup_split_list, sup_packs, eggs_supply, demand, demand_indices, split_list

    # Supply check    
    vec_sum = np.array([np.sum(vec[sup_indices[i]]* sup_packs[i]) 
                for i in np.arange(len(sup_indices))], dtype=np.int64)
    vec_sum_split = np.split(vec_sum, sup_split_list)[:-1]
    farm_wise_totals = np.array([np.sum(vec_sum_split[i]) for i in range(len(vec_sum_split))], dtype=np.int64)
    supply_check = np.all(farm_wise_totals <= eggs_supply)    

    # Demand check
    split_array = np.split(demand_indices, split_list)[:-1]
    vec_sum = np.array([np.sum(vec[i]) for i in split_array])
    demand_check = np.all(vec_sum <= demand)   
    
    # Zero check
    zero_check = np.all(vec >= 0)

    return demand_check and zero_check and supply_check

In [18]:
feasible_vec(vec)

False

In [19]:
zero_vec = np.zeros(585)
feasible_vec(zero_vec)

True

In [20]:
@njit
def poss_val(index:int, val:int, vec: np.ndarray):
    ''' Returns True if the 'val' being placed in 
        'index' position of 'vec' meets 'demand' and 'supply' 
        constraints '''
    vec_copy = vec.copy()
    vec_copy[index]=val
    return feasible_vec(vec_copy)


In [21]:
poss_val(index=0, val=10, vec=zero_vec)

True

In [22]:
@njit
def get_available_demand(vec:np.ndarray, index:int) -> int:
    ''' Returns the available demand for an index in an vector'''
    
    global demand_indices, split_list

    split_array = np.split(demand_indices, split_list)[:-1]
    for cprod_indices, dem in zip(split_array, demand):
        if index in cprod_indices:
            vec_cprod = np.array([vec[i] for i in cprod_indices], dtype=np.int64)
            
            available_demand = dem - (np.sum(vec_cprod) - vec_cprod[np.where(cprod_indices==index)])
            avail_demand = np.maximum(0, available_demand)[0] # Don't return negative demand
    return avail_demand

In [23]:
get_available_demand(vec=vec, index=20)

0

In [24]:
get_available_demand(vec=zero_vec, index=20)

3000

In [25]:
@njit
def get_availble_supply(vec:np.ndarray, index:int) -> int:
    global sup_indices, sup_split_list, sup_packs, eggs_supply
    
    sup_inices_split = np.split(sup_indices, sup_split_list)
    sup_packs_split = np.split(sup_packs, sup_split_list)

    loc = np.array([i for i in range(len(sup_inices_split)) if index in sup_inices_split[i]], dtype=np.int64)[0]  
    row, _ = np.where(sup_inices_split[loc] == index)

    arr_copy = np.where(sup_inices_split[loc] == index, 0, sup_inices_split[loc])
    supplied = np.sum(np.array([np.sum(vec[arr_copy[arr_row_ind]] * sup_packs_split[loc][arr_row_ind]) 
                for arr_row_ind in np.arange(arr_copy.shape[0])], dtype=np.int64))

    available_eggs = eggs_supply[loc] - supplied
    avail_supply = (available_eggs/sup_packs_split[loc][row]).astype(np.int64)

    return np.maximum(0, avail_supply)[0]

In [26]:
get_availble_supply(vec=vec, index=320)

0

In [27]:
get_availble_supply(vec=zero_vec, index=320)

8333

In [37]:
@njit
def random_val(vec:np.ndarray, index: int) -> np.int64:
    available_supply = get_availble_supply(vec=vec, index=index)
    available_demand = get_available_demand(vec=vec, index=index)

    if available_supply != 0 and available_demand != 0:
        return np.random.randint(0, np.minimum(available_demand, available_supply))
    else: 
        return 0

In [29]:
random_val(vec=vec, index=0)

1980

In [30]:
sc.pack_random_eggs()

array([5537, 3343, 2778, 5524, 3347, 2781, 5565, 3331, 2774, 2771, 1676,
       1384, 2780, 1668, 1386, 2776, 1677, 1380, 2804, 1659, 1381, 2790,
       1653, 1393, 2791, 1672, 1377,  831,  501,  821,  507,  839,  496,
        500,  416,  833,  416,  813,  426], dtype=int64)

In [31]:
len(sc.get_optimising_tuples())


585

In [32]:
def random_instantiate_vec():
    global sc
    zero_vec = np.zeros(len(sc.get_optimising_tuples()))
    fprods = [fprod for farm in sc.farms.keys() for fprod in sc.graph.successors(farm)]
    fprod_pack_dict = {fprod:pack for fprod, pack in zip(fprods, sc.pack_random_eggs())}

    for fprod, pack in fprod_pack_dict.items():
        cprods = [cprod for cprod in sc.graph.successors(fprod)]
        dist = sum_to_num(pack, len(cprods))
        for cprod, qty in zip(cprods, dist):
            index = sc.get_indices_dict().get((fprod, cprod), -1)
            if poss_val(index=index, val=qty, vec=zero_vec):
                zero_vec[index]=qty
            else:
                r = random_val(vec= zero_vec, index = index)
                zero_vec[index]= r
    
    assert feasible_vec(vec=zero_vec), 'random_instantiate_vec() returned an unfeasible vec '
    return zero_vec

In [49]:
rand_vec = random_instantiate_vec()

In [124]:
def calculate_profit(vec:np.ndarray, supply_chain:SupplyChain):
    transport_costs = np.sum(vec * supply_chain.get_transport_cost())
    supply_cost = np.sum(vec * supply_chain.get_supply_costs())
    prices = supply_chain.get_price_per_product()
    sales = np.sum(prices * vec)        
    total_cost = transport_costs + supply_cost
    profit = np.round((sales - total_cost), decimals=3)
    return profit


In [125]:
calculate_profit(vec=rand_vec, supply_chain=sc)

342828.022

In [121]:
# Transport
trans_cost = np.sum(rand_vec* sc.get_transport_cost())
trans_cost

5129.05810546875

In [122]:
# Supply costs
supply_cost = np.sum(rand_vec * sc.get_supply_costs())
supply_cost

9126.92

In [118]:
# sales
sales = np.sum(rand_vec * sc.get_price_per_product())
sales

357084.0

In [123]:
sales -(trans_cost+supply_cost)

342828.02189453127

In [126]:
feasible_vec(rand_vec)

True